# Data Extraction
This section details the process of extracting question data from a PDF file and corresponding answer keys from a TSV file.

## Importing libraries

* Pandas for data manipulation
* NumPy for the NaN datatype
* PyPDF for its ability to read the pfd files, although it requieres post-processing of the extracted text
* sqlite3 to create load the clean data into `bir_warehouse.db`

In [1]:
import pandas as pd
import numpy as np
from pypdf import PdfReader
import sqlite3

## Selecting year and type of exam

This function, `compute_expected_rows`, takes the exam year and topic as input and returns metadata crucial for subsequent data cleaning and loading steps. This metadata includes the expected maximum number of rows in the final DataFrame, the number of questions per exam, and the acronym for the exam type.  The maximum number of rows is calculated based on the number of questions and options per question for each year, following a specific pattern:

*   2024-2021: 210 questions, 4 options each
*   2020-2019: 185 questions, 4 options each
*   2018-2015: 235 questions, 4 options each
*   2014-2012: 235 questions, 5 options each
*   2011-2004: 260 questions, 5 options each

This information is essential for validating data integrity during the ETL process.  For example, the expected row count helps verify that all questions and options have been correctly extracted and transformed. The exam type acronym aids in consistent file naming and data organization.

In [2]:
def compute_expected_rows(year: int, topic: str) -> tuple:
    """Computes and returns metadata about the exam for a given year and topic

    Args:
        year (int): The year of the exam
        topic (str): The topic of the exam

    Returns:
        tuple: Contains:
            - max_rows: The expected maximum number of rows in the DataFrame
            - num_options: The number of questions in the exam
            - exam_type_acronym: The acronym for the exam type

    """
    info_year_dict: dict = {
        2011: [260, 5],
        2014: [235, 5],
        2018: [235, 4],
        2020: [185, 4],
        2021: [210, 4],
    }
    save_name_dict: dict = {
        "BIOLOGÍA": "bir",
        "FARMACIA": "fir",
        "QUÍMICA": "qir",
        "MEDICINA": "mir",
    }
    max_rows: int = 0
    num_options: int = 0
    for target_year, info_list in info_year_dict.items():
        if year <= target_year:
            max_rows = (info_list[0] * info_list[1]) + info_list[0]
            num_options = info_list[1]
            break
    else:
        max_rows = 1050
        num_options = 4
    return max_rows, num_options, save_name_dict[topic]

### Add year of the exam and topic 

In [3]:
year: int = 2021
topic: str = "MEDICINA" # Select from: BIOLOGÍA, FARMACIA, QUÍMICA, MEDICINA
total_num_rows: int
num_questions: int
exam_acronym: str
total_num_rows, num_questions, exam_acronym = compute_expected_rows(year, topic)
save_format: str = f"../data/clean/clean_{exam_acronym}_{year}.csv"

In [4]:
add_list: list = [26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 44, 47, 71, 79, 83, 84, 101, 145, 161]

## Extracting Questions from Pdf file

This section focuses on extracting the textual content of the exam questions from the provided PDF file.

**Path Definition:**

The path to the PDF file is dynamically constructed using the provided `year` and `topic` variables. This ensures flexibility and allows the script to process different exam files without manual path adjustments.

Finally it stores the output in a DataFrame and the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [5]:
path: str = f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C.pdf"
with PdfReader(path) as pdf_file:
    full_text: list = []
    for n in range(2, len(pdf_file.pages)):
        page = pdf_file.pages[n]
        text: str = page.extract_text()
        full_text.extend(text.splitlines())
raw_exam_df: pd.DataFrame = pd.DataFrame(full_text, columns=["text"])
raw_exam_df.head(10)

,text
0,
1,- 1 -
2,1. Pregunta asociada a la imagen 1.
3,
4,"Varón de 65 años, sin antecedentes de interés,"
5,"que consulta por tos, dolor torácico, fiebre,"
6,sudoración nocturna y disnea. En la radiografía
7,de tórax se observa un derrame pleural unilate-
8,"ral izquierdo, por lo que se realiza una TC torá-"
9,cica que confirma el d errame pleural y donde


## Extracting Answers from .tsv file into a DataFrame

This section focuses on extracting the answer key data from the provided .tsv file and loading it into a Pandas DataFrame.

**Path Definition and Data Loading:**

The path to the .tsv file is dynamically constructed using the `year` and `topic` variables, mirroring the approach used for the PDF file. This ensures consistency and flexibility in processing different exam files. 

The `pd.read_table()` function is then used to read the .tsv file directly into a Pandas DataFrame. Finally, the first 10 rows of the resulting DataFrame are displayed for a quick preview of the loaded data.

In [6]:
raw_answers_df: pd.DataFrame = pd.read_table(f"../data/raw/type_1/Raw_Cuaderno_{year}_{topic}_0_C_Respuestas.tsv")
raw_answers_df.head(10)

,V,RC,V.1,RC.1,V.2,RC.2,V.3,RC.3,V.4,RC.4
0,1,3,43,3,85,4.0,127,1,169,2.0
1,2,3,44,2,86,3.0,128,3,170,3.0
2,3,1,45,1,87,3.0,129,4,171,1.0
3,4,4,46,1,88,2.0,130,3,172,4.0
4,5,1,47,2,89,1.0,131,3,173,4.0
5,6,3,48,4,90,2.0,132,1,174,4.0
6,7,2,49,4,91,3.0,133,2,175,1.0
7,8,3,50,1,92,3.0,134,4,176,3.0
8,9,2,51,4,93,1.0,135,1,177,2.0
9,10,2,52,2,94,3.0,136,3,178,4.0


# Data Transformation

## Exam pdf

### Removing white space

This section focuses on cleaning the raw text extracted from the PDF exam file, preparing it for further processing. This involves removing extraneous whitespace, handling empty lines, and resetting the DataFrame index.

**1. Removing Page Number Artifacts:**

The extracted text may contain page number artifacts, which are removed by filtering out rows containing the string "Página".

In [7]:
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Página")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains("Pagina")]
raw_exam_df = raw_exam_df[~raw_exam_df["text"].str.contains(topic)]

footer_list: list = [f"- {npag} -" for npag in range(0,35,1)]

for footer in footer_list:
    raw_exam_df["text"] = raw_exam_df["text"].str.replace(footer, "")

**2. Striping Leading/Trailing Whitespaces**

Leading and trailing whitespace characters are removed from each text entry using the .str.strip() method.

In [8]:
raw_exam_df = raw_exam_df["text"].str.strip()

**3. Handling Empty Lines**

Empty lines, represented as empty strings, are replaced with NaN (Not a Number) values. 

Then rows containing NaN values, representing empty lines, are removed from the DataFrame using .dropna()

In [9]:
raw_exam_df = raw_exam_df.replace("", np.nan)
raw_exam_df = raw_exam_df.dropna()

**4. Reseting DataFrame Index**

The DataFrame index is reset after removing rows, ensuring a contiguous index and dropping the old index.

In [10]:
raw_exam_df = raw_exam_df.reset_index(drop=True)

**5. Displays the first 10 rows of the DataFrame**

In [11]:
raw_exam_df.head(10)

0                  1. Pregunta asociada a la imagen 1.
1       Varón de 65 años, sin antecedentes de interés,
2        que consulta por tos, dolor torácico, fiebre,
3      sudoración nocturna y disnea. En la radiografía
4      de tórax se observa un derrame pleural unilate-
5    ral izquierdo, por lo que se realiza una TC torá-
6        cica que confirma el d errame pleural y donde
7         además se observa un engrosamiento pleural y
8      micronódulos subpleurales en el lóbulo superior
9     izquierdo. Ante este hallazgo se decide realizar
Name: text, dtype: object

In [12]:
raw_exam_df.tail(10)

2900        3. Enfermedad por reflujo gastro-esofágico no
2901                                             erosiva.
2902                4. Esofagitis grado B de Los Ángeles.
2903    210. Señale cuál de las sigu ientes NO es una ...
2904     tación usual de las metástasis leptomeníngeas de
2905                                      los carcinomas:
2906                           1. Radiculopatía cervical.
2907                                      2. Hemiparesia.
2908                    3. Neuropatía de pares craneales.
2909                   4. Encefalopatía por hidrocefalia.
Name: text, dtype: object

### Joining Truncated Lines

This section addresses the issue of truncated lines and multi-line questions or options within the extracted exam text. A custom function, `process_multi_line_str`, is used to concatenate these lines, ensuring that each question and option is presented as a single, coherent string.

**Function: `process_multi_line_str(df)`**

This function iterates through the DataFrame, performing two main tasks:

1.  **Joining Truncated Words:**
    * It checks if a line ends with a hyphen ("-"), indicating a word that has been truncated at the end of a line.
    * If a line is truncated, it removes the hyphen and appends the content of the following line to the current line.
    * The following line is then removed from the DataFrame, and the index is reset.

2.  **Joining Multi-Line Questions/Options:**
    * It checks if the first character of the next line can be converted to an integer. This is used as a heuristic to identify the start of a new question or option.
    * If the next line does not start with an integer (i.e., it's a continuation of the current question or option), it is appended to the current line, separated by a space.
    * The next line is then removed, and the index is reset.

In [13]:
def process_multi_line_str(df: pd.DataFrame) -> pd.DataFrame:
    """Function to handle truncated lines

    Args:
        df (pd.DataFrame): A pandas dataframe

    Returns:
        pd.DataFrame: Returns the DataFrame without truncated lines
    """
    i: int = 0
    while i < len(df) - 1:
        if i < len(df) -1  and df.iloc[i].endswith("-"):
            df.iloc[i] = df.iloc[i][:-1] + df.iloc[i + 1]
            df = df.drop(i + 1)
            df = df.reset_index(drop=True)
        else:
            i += 1
    n: int = 0
    while n < len(df) -1:
        if n + 1 < len(df):
            try:
                int(df.iloc[n + 1][0:1])
                n += 1
            except ValueError:
                df.iloc[n] = df.iloc[n] + " " + df.iloc[n + 1]
                df = df.drop(n +1)
                df = df.reset_index(drop= True)
        else:
            break
    return df

The process_multi_line_str function is applied to the cleaned raw_exam_df DataFrame, and the resulting DataFrame is stored in exam_df_concat. The first few rows of the concatenated DataFrame are displayed to verify the results.

In [14]:
exam_df_concat: pd.DataFrame = process_multi_line_str(raw_exam_df)
exam_df_concat.head(10)

0    1. Pregunta asociada a la imagen 1. Varón de 6...
1                  1. Infiltración por adenocarcinoma.
2            2. Inflamación aguda con abscesificación.
3                        3. Inflamación granulomatosa.
4                                      4. Mesotelioma.
5    2. Pregunta asociada a la imagen 2. Varón de 2...
6                                 1. Embolia pulmonar.
7    2. Traumatismo torácico con afectación de la a...
8    3. Necrosis quística de la media y síndrome de...
9    4. Síndrome dislipémico familiar con grave ate...
Name: text, dtype: object

### Validating and Correction Incorrect Rows

This section focuses on validating the number of rows in the cleaned DataFrame against the expected number and correcting any rows that do not conform to the expected format.

**1. Calculating Expected and Actual Incorrect Rows:**

We calculate the expected number of incorrect rows by subtracting the total expected rows from the current number of rows in the DataFrame. We then identify the actual number of incorrect rows by counting the rows that do not end with a period ("."), a colon (":") or interrogation sign ("?").

In [15]:
num_rows_incorrect_expected: int = len(exam_df_concat) - total_num_rows
num_rows_incorrect: int = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].count()
id_rows_incorrect: list = exam_df_concat[~exam_df_concat.str.endswith((".", ":", "?"))].index.to_list()
print(f"Number of expected incorrect rows = {num_rows_incorrect_expected}")
print(f"Number of rows not ending with '.', ':' or '?' = {num_rows_incorrect}")
print(id_rows_incorrect)

Number of expected incorrect rows = 23
Number of rows not ending with '.', ':' or '?' = 26
[55, 66, 92, 98, 99, 375, 384, 397, 443, 444, 594, 601, 674, 762, 763, 764, 810, 950, 967, 968, 1014, 1015, 1028, 1029, 1046, 1057]


**2. Function: print_rows_incorrrect(wrong_id, df):**

This function takes a list of incorrect row indices and the DataFrame as input. It then prints the row identified as incorrect along with the 2 rows before and 2 rows after, providing context to help determine how to fix the error.

In [16]:
def print_rows_incorrrect(wrong_id: list[int], df: pd.DataFrame) -> None:
    """Print +/- 2 rows for context

    Args:
        wrong_id (list): Generated list of wrong row_id
        df (pd.DataFrame): DataFrame corresponding with the worng_id list
    """
    for wid in wrong_id:
        print(f"Id to fix {wid}:")
        print(df.iloc[wid -2: wid +3])

**3. Function: fix_incorrect(num_row, df):**

This function takes a list of row indices to fix and the DataFrame. It iterates through the indices in reverse order to avoid index shifting problems. Each incorrect row is concatenated with the row immediately following it, and the subsequent row is then dropped. The DataFrame index is reset after each concatenation.

In [17]:
def fix_incorrect(num_row: list[int], df: pd.DataFrame) -> pd.DataFrame:
    """Fix the incorrect rows in the DataFrame

    Args:
        num_row (list): List of ids to fix
        df (pd.DataFrame): The DataFrame to fix

    Returns:
        pd.DataFrame: Fixed
    """
    num_row: list = sorted(num_row, reverse=True)
    for n in num_row:
        df.iloc[n] = df.iloc[n] + df.iloc[n + 1]
        df = df.drop(n + 1)
        df = df.reset_index(drop= True)
    return df

**4. Validation and Correction Logic:**

We compare the expected and actual number of incorrect rows. If they match, we proceed to fix the incorrect rows using the `fix_incorrect` function. Otherwise, we print the context of the incorrect rows and raise a warning, indicating that manual inspection and correction are required.

In [18]:
if num_rows_incorrect_expected == num_rows_incorrect:
    exam_df_fixed = fix_incorrect(id_rows_incorrect, exam_df_concat)
    df_correct_rows: bool = len(exam_df_fixed) == num_rows_incorrect_expected
else:
    print_rows_incorrrect(id_rows_incorrect, exam_df_concat)
    # raise Warning("More incorrect rows that expected, uncomment the lines below and add ids to the list, and comment this line")
    rows_fix: list[int] = [55, 66, 92, 98, 99, 375, 384, 397, 443, 444, 594, 601, 762, 763, 764, 810, 950, 967, 968, 1014, 1015, 1046, 1057]
    exam_df_fixed: pd.DataFrame = fix_incorrect(rows_fix, exam_df_concat)

Id to fix 55:
53                    3. Retinitis por citomegalovirus.
54           4. Trombosis de vena central de la retina.
55    12. Pregunta asociad a a la imagen 12. Varón d...
56    1 paquete de cigarrillos al día desde hace 30 ...
57    1. Carcinoma transglótico derecho con adenopat...
Name: text, dtype: object
Id to fix 66:
64                                         3. Neumonía.
65                                    4. Hernia hiatal.
66    14. Pregunta asociad a a la imagen 14. Varón d...
67    92,9 mg/L y dímero D 1.630 ng/ml (normal <500)...
68    1. Solicitar angioTC  de arterias pulmonares p...
Name: text, dtype: object
Id to fix 92:
90                         3. Faringitis abscesificada.
91            4. Quiste dermoide lingual abscesificado.
92    19. Pregunta asociad a a la imagen 19. Varón d...
93    6 meses, así como diarrea frecuente. No ha com...
94                           1. Toxoplasmosis cerebral.
Name: text, dtype: object
Id to fix 98:
96          3. Leucoencefa

**5. Verification of Corrected Rows:**

We print the expected total number of rows and the actual number of rows in the corrected DataFrame to verify the results of the correction process.

In [19]:
print(f"Correct number of rows = {total_num_rows}")
print(exam_df_fixed.shape[0])

Correct number of rows = 1050
1050


### Pivoting the DataFrame

This section focuses on transforming the cleaned exam data into a structured format suitable for analysis. The DataFrame is pivoted to create columns for each question and its corresponding options.

**1. Preparing the DataFrame for Pivoting:**

The `exam_df_fixed` Series is converted to a DataFrame, and a 'group' column is created to identify each question and its options. An 'option_num' column is then generated to enumerate the options within each group.


In [20]:
exam_df_fixed = exam_df_fixed.to_frame()
groups: pd.Series = pd.Series((exam_df_fixed.index // (num_questions+1)) +1)
exam_df_fixed["group"] = groups  
exam_df_fixed["option_num"] = exam_df_fixed.groupby("group").cumcount() + 1

**2. Pivoting the DataFrame:**

The DataFrame is pivoted using the 'group' column as the index, the 'option_num' column as the columns, and the 'text' column as the values. The index is then reset.

In [21]:
exam_df_pivot: pd.DataFrame = exam_df_fixed.pivot(index="group", columns="option_num", values="text")
exam_df_pivot = exam_df_pivot.reset_index()

**3. Renaming Columns:**

The columns are renamed to more descriptive names, such as "Question", "Option_1", "Option_2", etc.

In [22]:
key_list: list = [x for x in range(1, num_questions+2, 1)]
val_list: list = ["Question"] + [f"Option_{x}" for x in range(1, 4 + 1, 1)]

exam_df_pivot = exam_df_pivot.rename_axis(None, axis=1).rename(columns=dict(zip(key_list, val_list)))

**4. Dropping the 'group' Column:**

The 'group' column is dropped as it is no longer needed

In [23]:
exam_df_pivot = exam_df_pivot.drop(columns=["group"])

**5. Displaying the Pivoted DataFrame:**

The first 10 rows of the pivoted DataFrame are displayed to verify the results of the transformation.

In [24]:
exam_df_pivot.head(10)

,Question,Option_1,Option_2,Option_3,Option_4
0,1. Pregunta asociada a la imagen 1. Varón de 6...,1. Infiltración por adenocarcinoma.,2. Inflamación aguda con abscesificación.,3. Inflamación granulomatosa.,4. Mesotelioma.
1,2. Pregunta asociada a la imagen 2. Varón de 2...,1. Embolia pulmonar.,2. Traumatismo torácico con afectación de la a...,3. Necrosis quística de la media y síndrome de...,4. Síndrome dislipémico familiar con grave ate...
2,3. Pregunta asociada a la imagen 3. Mujer de 8...,1. Adenocarcinoma ductal.,2. Cistadenoma seroso.,3. Neoplasia mucinosa quística.,4. Pancreatitis autoinmune asociada a enfermed...
3,4. Pregunta asociada a la imagen 4. Se realizó...,1. La esteatosis es menos intensa en la segund...,2. En la segunda biopsia no hay datos de infla...,3. La segunda biopsia muestra una situación es...,4. La presencia de fibrosis en puentes demuest...
4,5. Pregunta asociada a la imagen 5. Mujer de 9...,1. Carcinoma de células escamosas cutáneo inva...,2. Carcinoma basocelular esclerodermiforme.,3. Sarcoma de Kaposi.,4. Lupus vulgar.
5,6. Pregunta asociada a la imagen 6. M u j e r ...,1. Linfoma de Hodgkin cutáneo.,2. Síndrome de Sézary.,3. Linfoma no Hodgkin cutáneo tipo T.,4. Psoriasis pustulosa.
6,7. Pregunta asociada a la imagen 7. Niño de 23...,1. Punto de sangrado de origen gástrico.,2. Divertículo de Meckel revestido de mucosa g...,3. Punto de sangrado de origen genitourinario.,4. Gammagrafía normal.
7,8. Pregunta asociada a la imagen 8. Varón de 8...,1. Infarto agudo de miocardio.,2. Gammagrafía cardiaca normal.,3. Amiloidosis cardiaca mediada por transtiret...,4. Derrame pericárdico.
8,9. Pregunta asociada a la imagen 9. Mujer de 8...,1. La fiebre o febrícula es un síntoma present...,"2. Cuando se diagnostica la enfermedad, los ha...",3. Atendiendo a los hallazgos de la PET/TC en ...,4. La PET/TC es la técnica más específica para...
9,10. Pregunta asociad a a la imagen 10. Mujer d...,1. Fecundación in vitro.,2. Inseminación artificial.,3. Salpinguectomía bilateral laparoscópica.,4. Cromopertubación laparoscópica.


## Answers tsv

This section focuses on cleaning and consolidating the answer key data from the TSV file. The original data contains duplicate columns, which are processed and combined into a single, clean DataFrame.

**1. Defining Column Lists:**

We define lists containing the original column names and the duplicate column names.

In [25]:
original_col: list[str] = ["V", "RC"]
duplicate_cols: list[str] = ["V.1", "RC.1", "V.2", "RC.2", "V.3", "RC.3", "V.4", "RC.4"]

**2. Processing Duplicate Columns:**

We iterate through the duplicate_cols list in pairs, extracting the corresponding columns, renaming them to the original column names, and appending them to a list of DataFrames.

In [26]:
answers_df_list: list[pd.DataFrame] = [raw_answers_df[original_col]]

for i in range(0, len(duplicate_cols), 2):
    pair_cols: list = duplicate_cols[i:i+2]
    df_pair_col: pd.DataFrame = raw_answers_df[pair_cols].rename(columns={pair_cols[0]: "V", pair_cols[1]: "RC"})
    answers_df_list.append(df_pair_col)

**3. Concatenating DataFrames:**

The list of DataFrames is concatenated into a single DataFrame, ignoring the original index.

In [27]:
answers_df_clean: pd.DataFrame = pd.concat(answers_df_list, ignore_index=True)

**4. Displaying the Cleaned DataFrame:**

The cleaned DataFrame is printed to verify the results of the consolidation process.

In [28]:
answers_df_clean.head(10)

,V,RC
0,1,3.0
1,2,3.0
2,3,1.0
3,4,4.0
4,5,1.0
5,6,3.0
6,7,2.0
7,8,3.0
8,9,2.0
9,10,2.0


## Joining the DataFrames

This section focuses on merging the processed exam questions DataFrame with the cleaned answers DataFrame, performing final data cleaning, and verifying the integrity of the joined data.

**1. Joining the DataFrames:**

The pivoted exam questions DataFrame (`exam_df_pivot`) and the cleaned answers DataFrame (`answers_df_clean`) are joined horizontally using `pd.concat`. The 'V0' column, which represents the question number from the answers DataFrame, is then dropped. The first 10 rows of the joined DataFrame are displayed.

In [29]:
processed_exam_df: pd.DataFrame = pd.concat([exam_df_pivot, answers_df_clean], axis=1)
processed_exam_df = processed_exam_df.drop(columns=["V"])
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC
0,1. Pregunta asociada a la imagen 1. Varón de 6...,1. Infiltración por adenocarcinoma.,2. Inflamación aguda con abscesificación.,3. Inflamación granulomatosa.,4. Mesotelioma.,3.0
1,2. Pregunta asociada a la imagen 2. Varón de 2...,1. Embolia pulmonar.,2. Traumatismo torácico con afectación de la a...,3. Necrosis quística de la media y síndrome de...,4. Síndrome dislipémico familiar con grave ate...,3.0
2,3. Pregunta asociada a la imagen 3. Mujer de 8...,1. Adenocarcinoma ductal.,2. Cistadenoma seroso.,3. Neoplasia mucinosa quística.,4. Pancreatitis autoinmune asociada a enfermed...,1.0
3,4. Pregunta asociada a la imagen 4. Se realizó...,1. La esteatosis es menos intensa en la segund...,2. En la segunda biopsia no hay datos de infla...,3. La segunda biopsia muestra una situación es...,4. La presencia de fibrosis en puentes demuest...,4.0
4,5. Pregunta asociada a la imagen 5. Mujer de 9...,1. Carcinoma de células escamosas cutáneo inva...,2. Carcinoma basocelular esclerodermiforme.,3. Sarcoma de Kaposi.,4. Lupus vulgar.,1.0
5,6. Pregunta asociada a la imagen 6. M u j e r ...,1. Linfoma de Hodgkin cutáneo.,2. Síndrome de Sézary.,3. Linfoma no Hodgkin cutáneo tipo T.,4. Psoriasis pustulosa.,3.0
6,7. Pregunta asociada a la imagen 7. Niño de 23...,1. Punto de sangrado de origen gástrico.,2. Divertículo de Meckel revestido de mucosa g...,3. Punto de sangrado de origen genitourinario.,4. Gammagrafía normal.,2.0
7,8. Pregunta asociada a la imagen 8. Varón de 8...,1. Infarto agudo de miocardio.,2. Gammagrafía cardiaca normal.,3. Amiloidosis cardiaca mediada por transtiret...,4. Derrame pericárdico.,3.0
8,9. Pregunta asociada a la imagen 9. Mujer de 8...,1. La fiebre o febrícula es un síntoma present...,"2. Cuando se diagnostica la enfermedad, los ha...",3. Atendiendo a los hallazgos de la PET/TC en ...,4. La PET/TC es la técnica más específica para...,2.0
9,10. Pregunta asociad a a la imagen 10. Mujer d...,1. Fecundación in vitro.,2. Inseminación artificial.,3. Salpinguectomía bilateral laparoscópica.,4. Cromopertubación laparoscópica.,2.0


**2. Identifying and Handling Missing Values:**

Missing values are then replaced with 0.

In [30]:
processed_exam_df = processed_exam_df.fillna(0)

**3. Verifying Missing Values and Data Types:**

The number of missing values in each column is checked, and the data types of the columns are displayed. The 'RC' (correct answer) column is then converted to integer type.

In [31]:
processed_exam_df.isnull().sum()
processed_exam_df["RC"] = processed_exam_df["RC"].astype(int)
processed_exam_df.dtypes

Question    object
Option_1    object
Option_2    object
Option_3    object
Option_4    object
RC           int32
dtype: object

**4. Adding year column**

A column containing the year of the exam is added to the DataFrame.

In [32]:
processed_exam_df["year"] = year

**5. Displaying the Final DataFrame:**

The first 10 rows of the final joined and cleaned DataFrame are displayed.

In [33]:
processed_exam_df.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,1. Pregunta asociada a la imagen 1. Varón de 6...,1. Infiltración por adenocarcinoma.,2. Inflamación aguda con abscesificación.,3. Inflamación granulomatosa.,4. Mesotelioma.,3,2021
1,2. Pregunta asociada a la imagen 2. Varón de 2...,1. Embolia pulmonar.,2. Traumatismo torácico con afectación de la a...,3. Necrosis quística de la media y síndrome de...,4. Síndrome dislipémico familiar con grave ate...,3,2021
2,3. Pregunta asociada a la imagen 3. Mujer de 8...,1. Adenocarcinoma ductal.,2. Cistadenoma seroso.,3. Neoplasia mucinosa quística.,4. Pancreatitis autoinmune asociada a enfermed...,1,2021
3,4. Pregunta asociada a la imagen 4. Se realizó...,1. La esteatosis es menos intensa en la segund...,2. En la segunda biopsia no hay datos de infla...,3. La segunda biopsia muestra una situación es...,4. La presencia de fibrosis en puentes demuest...,4,2021
4,5. Pregunta asociada a la imagen 5. Mujer de 9...,1. Carcinoma de células escamosas cutáneo inva...,2. Carcinoma basocelular esclerodermiforme.,3. Sarcoma de Kaposi.,4. Lupus vulgar.,1,2021
5,6. Pregunta asociada a la imagen 6. M u j e r ...,1. Linfoma de Hodgkin cutáneo.,2. Síndrome de Sézary.,3. Linfoma no Hodgkin cutáneo tipo T.,4. Psoriasis pustulosa.,3,2021
6,7. Pregunta asociada a la imagen 7. Niño de 23...,1. Punto de sangrado de origen gástrico.,2. Divertículo de Meckel revestido de mucosa g...,3. Punto de sangrado de origen genitourinario.,4. Gammagrafía normal.,2,2021
7,8. Pregunta asociada a la imagen 8. Varón de 8...,1. Infarto agudo de miocardio.,2. Gammagrafía cardiaca normal.,3. Amiloidosis cardiaca mediada por transtiret...,4. Derrame pericárdico.,3,2021
8,9. Pregunta asociada a la imagen 9. Mujer de 8...,1. La fiebre o febrícula es un síntoma present...,"2. Cuando se diagnostica la enfermedad, los ha...",3. Atendiendo a los hallazgos de la PET/TC en ...,4. La PET/TC es la técnica más específica para...,2,2021
9,10. Pregunta asociad a a la imagen 10. Mujer d...,1. Fecundación in vitro.,2. Inseminación artificial.,3. Salpinguectomía bilateral laparoscópica.,4. Cromopertubación laparoscópica.,2,2021


### Filtering by relevant questions

**1. Filtering for id**

A list of the number of question is used to filter the DataFrame, but first doing a -1 so it matches with the id. 

Finally the indexes are reseted

In [34]:
id_filter = [x-1 for x in add_list]
processed_exam_df_filtered = processed_exam_df.iloc[id_filter]
processed_exam_df_filtered = processed_exam_df_filtered.reset_index(drop= True)
print(f"Expected number of rows = {len(id_filter)}\nNumber of rows in filtered df = {processed_exam_df_filtered.shape[0]}")

Expected number of rows = 19
Number of rows in filtered df = 19


**2. Showing the filtered DataFrame**

Using head and tail

In [35]:
processed_exam_df_filtered.head(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,26. ¿Cuál de los siguientes estudios de la coa...,1. Tiempo de tromboplastina parcial activado.,2. Tiempo de trombina.,3. Tiempo de protrombina.,4. Determinación de factor von Willebrand.,3,2021
1,"27. En relación con la toxicidad del etanol, s...",1. El acetato producido en el metabolismo hepá...,2. El incremento en la relación NADH/NAD+ en e...,3. El ácido úrico compite con el lactato en la...,4. Los principales marcadores bioquímicos de i...,2,2021
2,29. En relación con el control fisiológico hor...,1. El óxido nítrico derivado del endotelio aum...,2. Las prostaglandinas y la bradicinina reduce...,3. La noradrenalina y la adrenalina contraen l...,4. La angiotensina II contrae preferentemente ...,1,2021
3,30. Con respecto a la actividad eléctrica del ...,1. El ritmo de casi todas las contracciones ga...,2. La despolarización de las ondas lentas vien...,3. Los potenciales en espiga son potenciales d...,4. En la generación y características de los p...,2,2021
4,31. En relación con el volumen y flujo sanguín...,1. En la espiración forzada se puede generar t...,2. Cuando disminuye la concentración de oxígen...,3. El ejercicio aumenta el flujo sanguíneo en ...,4. El aumento del gasto cardiaco durante el ej...,2,2021
5,32. En cuanto a las características de los pot...,1. El potencial de membrana de reposo tiene un...,2. La duración del potencial de acción de la f...,3. La velocidad de conducción del potencial de...,4. El flujo de corriente en la profundidad de ...,4,2021
6,33. Las diferentes clases ( isotipos) de inmun...,1. La IgA se une al receptor poli-Ig (pIgR) pa...,2. La IgG se une al receptor III para Fc-gamma...,3. La IgM se une a C1q y activa la vía clásica...,4. La IgE se une al receptor para Fc-épsilon (...,1,2021
7,34. En la inmunoterapia antitumoral basada en ...,1. La molécula PD-1 es un receptor inhibidor d...,2. La interacción de PD-1 y sus ligandos inhib...,3. El bloqueo de PD-1 o de sus ligandos potenc...,4. El uso de este tipo de inmunoterapia no se ...,3,2021
8,35. Diferentes mutaciones en el gen AIRE (Auto...,"1. Está ligado al cromosoma X, por lo que el s...",2. Se expresa en los timocitos que no expresan...,3. S e e x pr e s a e n l a s c é l ul a s ...,4. Se expresa en células epiteliales de la méd...,4,2021
9,3 7 . R e s p e c t o a l a s c é l u l a ...,1. Colaboran en la eliminación de células tumo...,2. Expresan el correceptor inhibidor CTLA-4.,3. Coexpresan tanto CD4 como CD8.,4. Liberan citoquinas como IL-17 o IL-22.,2,2021


In [36]:
processed_exam_df_filtered.tail(10)

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
9,3 7 . R e s p e c t o a l a s c é l u l a ...,1. Colaboran en la eliminación de células tumo...,2. Expresan el correceptor inhibidor CTLA-4.,3. Coexpresan tanto CD4 como CD8.,4. Liberan citoquinas como IL-17 o IL-22.,2,2021
10,44. La especificidad de una prueba diagnóstica...,"1. De cada 100 resultados negativos, 94 corres...","2. De cada 100 pacientes sanos, en 94 el resul...","3. De cada 100 pacientes enfermos, en 6 el res...","4. De cada 100 resultados positivos, 6 corresp...",2,2021
11,47. ¿En cuál de las siguientes infecciones se ...,1. Varicela.,2. Fiebres hemorrágicas virales.,3. Norovirus.,4. Clostridioides difficile.,2,2021
12,71. En relación con el estudio prenatal de ane...,1. El valor de la fracción libre de la gonadot...,2. Se realiza mediante la combinación de datos...,3. Está indicada en el primer trimestre en pob...,4. La transonolucencia nucal debe medirse entr...,2,2021
13,79. En caso de que haya manifestaciones clínic...,1. Meningitis.,2. Genital.,3. Neumonía.,4. Gingivoestomatitis y faringitis.,4,2021
14,83. La fenilcetonuria es una enfermedad metabó...,1. La fenilalanina debe suprimirse en la dieta...,2. La discapacidad intelectual es constante y ...,3. Puede deberse a un déficit de tetrahidrobio...,4. S e r e c o m i e n d a s u e s t u d i ...,1,2021
15,"84. En una enfermedad genética neurológica, el...",1. Atrofia muscular espinal.,2. Distrofia muscular de Duchenne.,3. Enfermedad de Parkinson.,4. Distrofia miotónica de Steinert.,4,2021
16,101. Todas las siguientes enfermedades neurode...,1. Enfermedad de Alzheimer.,2. Demencia frontotemporal.,3. Enfermedad de Parkinson.,4. Parálisis supranuclear progresiva.,3,2021
17,145. ¿Cuál de las siguientes vacunas NO está r...,1. Vacuna antineumococo PPV 23.,2. Vacuna antihaemophilus influenza B.,3. Vacuna antimeningocócica C.,4. Vacuna antitetánica.,4,2021
18,161. En el diagnóstico de un proceso linfoprol...,1. Linfoma folicular.,2. Linfoma de la zona marginal.,3. Linfoma linfoblástico.,4. Linfoma del manto.,4,2021


# Data Loading

## Saving in csv file

The `processed_exam_df_filtered` DataFrame, containing the cleaned and transformed exam data, is saved to a CSV file with the name generatd in the first step in the "data" directory. The `index=False` argument ensures that the DataFrame index is not written to the file, resulting in a cleaner output. This CSV file can be used for further analysis or as input for other applications.

In [37]:
processed_exam_df_filtered.to_csv(save_format, index= False)

## Writing into data warehouse

This section details the process of loading the cleaned and transformed exam data into an SQLite database, serving as a data warehouse for further analysis and querying.

**1. Database Connection and Path:**

The path to the SQLite database file is defined, and a connection is established using the `sqlite3` library.

In [38]:
db_path: str = "../data/clean/bir_warehouse.db"

**2. SQL Queries Definition:**

Two SQL queries are defined: one for inserting questions into the questions table and another for inserting options into the questions_options table.

In [40]:
query_question: str = """
INSERT INTO questions (exam_year, exam_subject, question) 
VALUES((SELECT id_year FROM year WHERE year_name = ?),
    (SELECT id_type FROM exam WHERE exam_type = ?),
    ?);
"""

query_options: str = """
INSERT INTO questions_options (question_id, option_num, option_text, is_correct)
VALUES((SELECT q.id 
        FROM questions AS q
        JOIN year AS y ON q.exam_year = y.id_year
        WHERE q.question = ? AND y.year_name = ?),
    ?,
    ?,
    ?)
"""

**3. Database Interaction and Data Insertion:**

A with statement is used to establish a connection to the SQLite database, ensuring that the connection is properly closed after use. A cursor is created, and the processed_exam_df DataFrame is iterated over to insert each question and its options

Some spacing issues were fixed in the csv file, so the DataFrame will be loaded from the csv file

In [41]:
processed_exam_df_filtered = pd.read_csv("../data/clean/clean_mir_2021.csv")
processed_exam_df_filtered.head()

,Question,Option_1,Option_2,Option_3,Option_4,RC,year
0,26. ¿Cuál de los siguientes estudios de la coa...,1. Tiempo de tromboplastina parcial activado.,2. Tiempo de trombina.,3. Tiempo de protrombina.,4. Determinación de factor von Willebrand.,3,2021
1,"27. En relación con la toxicidad del etanol, s...",1. El acetato producido en el metabolismo hepá...,2. El incremento en la relación NADH/NAD+ en e...,3. El ácido úrico compite con el lactato en la...,4. Los principales marcadores bioquímicos de i...,2,2021
2,29. En relación con el control fisiológico hor...,1. El óxido nítrico derivado del endotelio aum...,2. Las prostaglandinas y la bradicinina reduce...,3. La noradrenalina y la adrenalina contraen l...,4. La angiotensina II contrae preferentemente ...,1,2021
3,30. Con respecto a la actividad eléctrica del ...,1. El ritmo de casi todas las contracciones ga...,2. La despolarización de las ondas lentas vien...,3. Los potenciales en espiga son potenciales d...,4. En la generación y características de los p...,2,2021
4,31. En relación con el volumen y flujo sanguín...,1. En la espiración forzada se puede generar t...,2. Cuando disminuye la concentración de oxígen...,3. El ejercicio aumenta el flujo sanguíneo en ...,4. El aumento del gasto cardiaco durante el ej...,2,2021


In [44]:
with sqlite3.connect(db_path) as bir_warehouse:
    cur = bir_warehouse.cursor()
    for question in processed_exam_df_filtered.itertuples():
        cur.execute(query_question, (str(question[7]), exam_acronym, question[1]))
        bir_warehouse.commit()
        for n, option in enumerate(question[2:6]):
            cur.execute(query_options, (question[1], str(year), n+1, option, n+1 == question[6]))
            bir_warehouse.commit()